# Principles of Digital Communication - Project
_Harold Benoit, Tom Demont_

_May 2021_

In [768]:
import random

import numpy as np

## 1. Text conversion from string to bits
We will first create the utility methods for conversion from utf-8 text.

In [769]:
def string_to_bitarray(string):
    string_bytes = string.encode('utf-8')
    bits = []
    for b in string_bytes:
        mask = 1
        for i in range(7,-1,-1):
            bits.append((b >> i)&mask)
    return bits

In [770]:
print(string_to_bitarray('Hello World 😃'))

[0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1]


In [771]:
def bitarray_to_int(bitarray):
    # the format for putting one bit in a string
    string = "{0:0"+str(1)+"b}"
    integer = ''
    for bit in bitarray:
        integer += string.format(bit)
    return int(integer,2)

In [772]:
print(bitarray_to_int([1,1,0]))

6


In [773]:
def bitarray_to_string(bitarray):
    bytes = []
    # the format for putting one bit in a string
    string = "{0:0"+str(1)+"b}"

    # go over all bytes given
    for nb_bytes in range(0, len(bitarray)//8):
        # recreates the byte from the bits
        byte = ''
        for bit in range(0,8):
            byte += string.format(bitarray[nb_bytes*8+bit])
        # adds the integer associated to the binary string in that byte (between 0 and 255)
        bytes.append(int(byte,2))
    return str(bytearray(bytes), 'utf-8')

#### Testing our system
We verify that transforming a string to a bit array back and forth works well:

In [774]:
bitarray_to_string(string_to_bitarray('Hello World 😃'))

'Hello World 😃'

## 2. Recover from channel erasure
To recover from channel erasure, we'll implement a **parity check**.

### Encoding for channel erasure

Per every 2 uses of the channel, we'll send a third parity check codeword. It is generated from applying xor operator
 on the 2 first encoded blocks bits and sending the associated codeword.

For example, when $k=3$, suppose we want to send $001$ and $010$. $001$ is encoded by $c_1$ and $010$ is encoded by
$c_2$. Our parity codeword will then be $001 \oplus 010=011$, encoded by $c_3$.

In [775]:
def compute_parity_check(bitarray_1, bitarray_2):
    parity_check = []
    for bit in range(0,len(bitarray_1)):
        parity_check.append(bitarray_1[bit] ^ bitarray_2[bit])
    return parity_check

In [776]:
print("Hello        ",string_to_bitarray('Hello'))
print("World        ",string_to_bitarray('World'))
print("Parity check ",compute_parity_check(string_to_bitarray('Hello'),string_to_bitarray('World')))

Hello         [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1]
World         [0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
Parity check  [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1]


Also, we'll interleave our codewords so that, erasing one channel symbol over 3 will in fact erase a full codeword.
We'll recover this one with the parity check by inverting our xor operation.

For example, if we want to send $c_1c_2c_3$, we'll send $c_{10}c_{20}c_{30}c_{11}c_{21}c_{31}...c_{1n}c_{2n}c_{3n}$

In [777]:
def interleave_codewords_to_send(codeword_1, codeword_2, parity_check_encoded):
    codewords = np.array([codeword_1, codeword_2, parity_check_encoded])
    return codewords.transpose().flatten()

In [778]:
# Tests interleaving 'Hello', 'World' and their parity check
interleaved_hello_world = interleave_codewords_to_send(string_to_bitarray('Hello'), string_to_bitarray('World'),
                                           compute_parity_check
(string_to_bitarray('Hello'),string_to_bitarray('World')))
print(interleaved_hello_world)

[0 0 0 1 1 0 0 0 0 0 1 1 1 0 1 0 1 1 0 1 1 0 1 1 0 0 0 1 1 0 1 1 0 0 0 0 0
 1 1 1 1 0 0 1 1 1 1 0 0 0 0 1 1 0 1 1 0 0 1 1 1 0 1 1 0 1 0 1 1 0 0 0 0 0
 0 1 1 0 1 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 1
 1 1 0 1 0 1 1 0 1]


### Making an erasing channel
We build the channel that randomly removes $1/3$ of the coordinates given

In [779]:
# the channel input is the array of interleaved codewords and their parity check
def erasing_channel(chan_input):
    chan_input = np.clip(chan_input,-1,1)
    erased_index = np.random.randint(3)
    chan_input[erased_index:len(chan_input):3] = 0
    return chan_input

### Build a simple codewords book for erasure testing
The codebook will be as follows: $0 \to -1$ and $1 \to 1$. This codebook is very simple, has $k=n=1$ and will just be
 useful for making our bitstring able to go through the erasing channel.

In [780]:
def encode(bitarray):
    encoded = []
    for bit in bitarray:
        if bit == 0:
            encoded.append(-1)
        elif bit == 1:
            encoded.append(1)
        else:
            print("bit array should only contain 0 or 1")
            return
    return encoded

In [781]:
print("Hello binary:  ",string_to_bitarray('Hello'))
print("Hello encoded: ",encode(string_to_bitarray('Hello')))

Hello binary:   [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1]
Hello encoded:  [-1, 1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1, 1, 1]


### Decoding procedure
We'll :
1. Uninterleave the received array to retrieve what should be associated to each codeword
2. Find the codeword that was erased by looking for a $[0,...,0]$ array
3. Decode the codewords that was correctly sent
  * If the erased codeword is the third one, it means that only the parity check was affected which is not a problem,
   we just return the decoded bits from both first codewords
   * If the erased codeword is one of both first, we apply $\oplus$ on both correctly decoded bits to retrieve the
   erased
    one thanks to the parity check

In [782]:
# the triplet contains 2 codewords encoding k bits and the codeword encoding the parity check bits
def uninterleave_codewords_received(three_codewords):
    # we create chunks of size 3, the interleaved codewords matrix (shape (n,3))
    codewords_triplet = np.array_split(np.array(three_codewords), len(three_codewords)//3)

    # we transpose it to retrieve the aligned codewords in order
    codewords = np.array(codewords_triplet).transpose().flatten()
    return (codewords)

In [783]:
# gives both correctly decoded codewords and the value of H observed (the modulus class of erased coordinate)
def decode_output_retrieve_erased(uninterleaved_three_codewords):
    decoded = []
    erased = -1
    n = len(uninterleaved_three_codewords)//3

    # classic decoding from our codebook
    for i,symbol in enumerate(uninterleaved_three_codewords):
        if symbol == 1:
            decoded.append(1)
        elif symbol == -1:
            decoded.append(0)
        elif symbol == 0:
            # this one was erased
            erased = (i//n)%3
        else:
            print("Error: symbols cannot be other than 0 and ±1")
            return
    return decoded,erased

In [784]:
# decoded contains both correctly decoded codewords (can be information codeword and parity or 2 information codewords)
# the parity codeword (present if erased != 2) is in the last half of the decoded bit array
def restore_erased(decoded, erased):
    if erased != 2:
        # an information codeword was erased
        restored = []
        n = len(decoded)//2
        for i in range(n):
            # we recompute the erased one by binary one time padding
            restored.append(decoded[i]^decoded[n+i])
        if erased == 0:
            # the first codeword was erased
            return np.array([restored,decoded[0:len(decoded)//2]]).flatten()
        elif erased == 1:
            # the second codeword was erased
            return np.array([decoded[0:len(decoded)//2],restored]).flatten()
    else:
        # the parity check codeword was erased
        return decoded

#### Testing our system
We now try to send "HelloWorld" message over the erasing channel and try to restore it:

In [785]:
channel_input = encode(interleaved_hello_world)
print("Channel input:    ", channel_input)
channel_output = erasing_channel(channel_input)
print("Channel output:   ", list(channel_output))

print()

uninterleaved_input = encode(np.array([string_to_bitarray('Hello'), string_to_bitarray('World'),compute_parity_check
(string_to_bitarray('Hello'),string_to_bitarray('World'))]).flatten())
print("Uninterleaved in: ", list(uninterleaved_input))
uninterleaved_output = uninterleave_codewords_received(channel_output)
print("Uninterleaved out:", list(uninterleaved_output))

print()

decoded_output, erased_codewords = decode_output_retrieve_erased(uninterleaved_output)
print("Decoded output:   ",decoded_output)
print("Erased codeword:  ",erased_codewords,"[3]")

print()

sent = np.array([string_to_bitarray('Hello'), string_to_bitarray('World')]).flatten()
print("Sent:             ",list(sent))
restored = restore_erased(decoded_output, erased_codewords)
print("Restored:         ",list(restored))
print("Restored string:  ",bitarray_to_string(restored))

Channel input:     [-1, -1, -1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, -1, 1]
Channel output:    [-1, 0, -1, 1, 0, -1, -1, 0, -1, -1, 0, 1, 1, 0, 1, -1, 0, 1, -1, 0, 1, -1, 0, 1, -1, 0, -1, 1, 0, -1, 1, 0, -1, -1, 0, -1, -1, 0, 1, 1, 0, -1, -1, 0, 1, 1, 0, -1, -1, 0, -1, 1, 0, -1, 1, 0, -1, -1, 0, 1, 1, 0, 1, 1, 0, 1, -1, 0, 1, -1, 0, -1, -1, 0, -1, 1, 0, -1, 1, 0, -1, -1, 0, -1, 1, 0, -1, 1, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, -1, 1, 0, -1, 1, 0, -1, -1, 0, -1, 1, 0, 1, 1, 0, -1, 1, 0, 1, 1, 0, 1]

Uninterleaved in:  [-1, 1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1, -1,

## 3. Recovering from noise
The problem states that each received symbol $Y_i=\tilde{X}_i+Z_i$ where $\tilde{X}_i$ is the symbol after passing
the **erasing channel** and $Z_i\sim \mathcal{N}(0,10)$ is the added **White Gaussian Noise**.
To deal with the erasures, we'll apply the same procedure as above (that works fine also on other codebooks).
However, the white noise need to be tackled smartly considering its high variance, and the low energy of channel input.

To do so, we choose as our codebook the **Orthogonal Code $C_k$** which has a high distance between codewords for
reasonable amount of energy and is pretty simple to generate.

We recall the recursive $C_k$ definition:
* $C_0=\{1\}$
* $C_{k+1}=C_k'$ where $C_k'$ is such that for every $c\in C_k$, $(c,c)$ and $(c,-c) \in C_k'$
* e.g.: $C_1=\{(1,1),(1,-1)\}$, $C_2=\{(1,1,1,1),(1,1,-1,-1),(1,-1,1,-1),(1,-1,-1,1)\}$ ...

In [786]:
def create_c_k(k):
    c_0 = 1
    c_k = [c_0]
    for i in range(1,k+1):
        c_k_1 = []
        for codeword in c_k:
            c_c = np.array([[codeword, codeword]]).flatten()
            c_min_c = np.array([[codeword, -1*codeword]]).flatten()
            c_k_1.append(c_c)
            c_k_1.append(c_min_c)
        c_k = c_k_1.copy()
    return np.array(c_k)

In [787]:
# we verify if c_2 was correctly computed
print(create_c_k(2))

[[ 1  1  1  1]
 [ 1  1 -1 -1]
 [ 1 -1  1 -1]
 [ 1 -1 -1  1]]


### Combine parity check and orthogonal coding
We now can create codewords associated to each k-plet of bits. Let's implement the encodng of a bitarray for a
parametrized value of k

In [788]:
# Encodes both arrays of k bits with the orthogonal code and return encoded 2k bits and their k parity check bits
# encoded in 3 codewords
def encode_ortho(bitarray_1, bitarray_2):
    k = len(bitarray_1)
    if len(bitarray_1)!=len(bitarray_2):
        print("Error, both encoded bitarrays should have same length (length k)")
        return
    # creates the indexes of those k bits (and their pcheck) in the codebook
    codeword_1_index = bitarray_to_int(bitarray_1)
    codeword_2_index = bitarray_to_int(bitarray_2)
    parity_check = compute_parity_check(bitarray_1, bitarray_2)
    codeword_check_index = bitarray_to_int(parity_check)

    codebook = create_c_k(k)

    return codebook[codeword_1_index],codebook[codeword_2_index],codebook[codeword_check_index]

We try to encode the $10$ and $01$ bits pairs. We then have $k=2$ and use $C_2$. Their parity check bits are $11$. We
 should then have the codewords $3$, $2$ and $4$ of $C_2$

In [789]:
for cw in encode_ortho([1,0],[0,1]):
    print(list(cw))

[1, -1, 1, -1]
[1, 1, -1, -1]
[1, -1, -1, 1]


### Simulate the erasing noisy channel
We create the channel that outputs $Y_i$. For testing purposes, we make it return the realization of the erased symbol

In [790]:
def noisy_erasing_channel(chan_input):
    chan_input = np.clip(chan_input,-1,1)
    erased_index = np.random.randint(3)
    chan_input[erased_index:len(chan_input):3] = 0
    return chan_input + np.sqrt(10)*np.random.randn(len(chan_input)), erased_index

### Decoding procedure
The decoding will happen following:
1. We uninterleave the received symbols
2. We compute the empirical mean of each codewords classes (the codewords placed $0^{th}[3]$, $1^{st}[3]$ and
$2^{nd}[3]$ in the codewords sequence)
    * The mean closest to 0 represents the class of erased codewords
3. For the unerased (but noisy) channel outputs, we compute their scalar product with every codeword of $C_k$
    * The highest product is the one with the most similar codewords, we interpret the noisy channel output as being
    this symbol
4. We decode the decided codewords and use them to compute (with parity check) the erased codeword

#### Finding the erased coordinate

In [791]:
def find_erased_noisy(uninterleaved_chan_output, k):
    n = 2**k
    # We divide out output in arrays of length n (each represent a codeword)
    output_symbols = np.array_split(np.array(uninterleaved_chan_output), len(uninterleaved_chan_output)//n)

    mean_0 = mean_1 = mean_2 = 0.
    for i,codeword in enumerate(output_symbols):
        # we are interested in computing the distance from 0 to mean
        # we have to sum the squared output symbols
        # (as codewords input have zero mean w/out absolute distance computation)
        if i%3 == 0:
            mean_0 += np.sum(codeword**2)
        if i%3 == 1:
            mean_1 += np.sum(codeword**2)
        if i%3 == 2:
            mean_2 += np.sum(codeword**2)

    # As all means should be computed by dividing by the same number, we don't make that extra computation that does
    # not changes the order
    if abs(mean_0) < abs(mean_1) and abs(mean_0) < abs(mean_2):
        return 0
    if abs(mean_1) < abs(mean_0) and abs(mean_1) < abs(mean_2):
        return 1
    if abs(mean_2) < abs(mean_1) and abs(mean_2) < abs(mean_0):
        return 2
    return np.random.randint(3)

We try to find the erased codeword when noise is added. We'll again use $C_2$ codebook for testing:

In [792]:
cw_1, cw_2, cw_3 = encode_ortho([1,0],[0,1])
print("Codeword 1:     ",list(cw_1))
print("Codeword 2:     ",list(cw_2))
print("Codeword 3:     ",list(cw_3))
interleaved_C2 = interleave_codewords_to_send(cw_1, cw_2, cw_3)

chan_output_1, erased_index = noisy_erasing_channel(interleaved_C2)

uninterleaved_C2 = uninterleave_codewords_received(chan_output_1)
print("Uninterleaved:  ",uninterleaved_C2)

erased_found = find_erased_noisy(uninterleaved_C2, 2)
print("Erased predict: ",erased_found)
print("Erased real:    ",erased_index)

Codeword 1:      [1, -1, 1, -1]
Codeword 2:      [1, 1, -1, -1]
Codeword 3:      [1, -1, -1, 1]
Uninterleaved:   [ 2.78821879 -7.06091086  2.86357522  1.30166846 -1.14177039  5.53459122
  3.12911533 -0.09558056 -2.66691314 -1.34220512 -6.36761939  2.52022946]
Erased predict:  1
Erased real:     1


We see that the prediction is pretty bad. Indeed, this method relies on the fact that a huge number of realizations
will reveal the global mean.

12 channel outputs is not enough to approximate the mean.

We try to send more codewords through our channel:

In [793]:
# defines a sending/receiving protocol and outputs the erased prediction and real values
def test_erasure_recovery(nb_triplets_to_send):
    interleaved_to_send = []
    for i in range(nb_triplets_to_send):
        cw_1, cw_2, cw_3 = encode_ortho([i%2,(i+1)%2],[(i+1)%2,i%2])
        interleaved_to_send.append(interleave_codewords_to_send(cw_1, cw_2, cw_3))
    interleaved_to_send = np.array(interleaved_to_send).flatten()

    chan_output_2, erased_real = noisy_erasing_channel(interleaved_to_send)

    uninterleaved_received = []
    # we split back our array in triplets_to_send (which makes it contains codewords triplets at each element)
    output_triplets = np.array_split(np.array(chan_output_2),nb_triplets_to_send)
    for triplet in output_triplets:
        uninterleaved_received.append(uninterleave_codewords_received(triplet))
    uninterleaved_received = np.array(uninterleaved_received).flatten()

    erased_found = find_erased_noisy(uninterleaved_received, 2)

    return erased_found,erased_real

In [794]:
nb_success=nb_fails=0

for test in range(100):
    erased_found_t,erased_real_t = test_erasure_recovery(1000)
    if erased_found_t == erased_real_t: nb_success+=1
    else: nb_fails+=1

print("Correctness ratio: ",nb_success/(nb_success+nb_fails))

Correctness ratio:  1.0


We finally observe that, using the channel for 3000 informations+parity check bits, we discover the erased coordinate
 with very high probability.

#### Denoising the information codewords